In [1]:
from cobra.io import load_model, load_json_model
import numpy as np
import pandas as pd

In [2]:
%run ../HelperFunctions_MOBO.ipynb

# iML1515

In [8]:
model_iML1515 = load_model("iML1515")

In [9]:
# M9 with essential trace metals
medium_iJO1366_reduced = {
    'EX_pi_e': 34.90, # in M9
    'EX_mn2_e': 0.001, # - required?; drops at 0.0001
    'EX_fe2_e': 0.1, # - required?; drops at 0.01
    'EX_glc__D_e': 10.0, # in M9
    'EX_zn2_e': 0.001, # - required?; drops at 0.0001
    'EX_mg2_e': 1.0, # in M9 
    'EX_ca2_e': 0.05, # in M9
    'EX_ni2_e': 0.001, # - required?; drops at 0.0001
    'EX_cu2_e': 0.001, # - required?; drops at 0.0001
    'EX_cobalt2_e': 0.0001, # - required; drops at 0.00001 
    'EX_mobd_e': 0.0005, # - required?; drops at 0.000001
    'EX_so4_e': 1.0, # in M9
    'EX_nh4_e': 9.3475, # in M9
    'EX_k_e': 11.02, # in M9
    'EX_na1_e': 52.038, # in M9
    'EX_cl_e': 13.6755, # in M9
    'EX_o2_e': 20.0, # in M9 II - drops at 10
}
bounds_iJO1366_reduced = {
    'EX_pi_e': (0.0, 50),
    'EX_mn2_e': (0.001, 0.001), # fix "trace" 
    'EX_fe2_e': (0.1, 0.1), # fix "trace"
    'EX_glc__D_e': (1.0, 10),
    'EX_zn2_e': (0.001, 0.001), # fix "trace"
    'EX_mg2_e': (0.0, 10),
    'EX_ca2_e': (0.0, 10),
    'EX_ni2_e': (0.001, 0.001), # fix "trace"
    'EX_cu2_e': (0.001, 0.001), # fix "trace"
    'EX_cobalt2_e': (0.0001, 0.0001), # fix "trace"
    'EX_mobd_e': (0.0005, 0.0005), # fix "trace"
    'EX_so4_e': (0.0, 10),
    'EX_nh4_e': (0.0, 10),
    'EX_k_e': (0.0, 20),
    'EX_na1_e': (0.0, 100.0), # fix - can be set to 0
    'EX_cl_e': (0.0, 20),
    'EX_o2_e': (0, 20), # fix - can't be set to 10 or lower
}
# costs are in £/mol
costs_iJO1366_reduced = {
    'EX_pi_e': 23.4234, # Phosphate - approximate (several sources)
    'EX_mn2_e': 0.0, #33.25, # Manganese - MnCl2·4H20
    'EX_fe2_e': 0.0, #37.5, # IronII - as iron sulfate FeSO4·7H2O
    'EX_glc__D_e': 7.7647236, # Glucose
    'EX_zn2_e': 0.0, #28.3, # Zinc - as Zn(CH3CHOOH)·H2O
    'EX_mg2_e': 19.1022, # Magnesium - as MgSO4·7H2O - approximate bc. half of 38.2044
    'EX_ca2_e': 18.08223, # Calcium - as CaCl2·2H2O
    'EX_ni2_e': 0.0, #53.24, # Nickel - as NiCl2·6H2O
    'EX_cu2_e': 0.0, #31.37, # Copper - CuCl2·2H2O
    'EX_cobalt2_e': 0.0, #114.39, # Cobalt - as CoCl2·6H2O
    'EX_mobd_e': 0.0, #184.12, # Molybdenum - molybdate NaMoO4·2H2O
    'EX_so4_e': 19.1022, # Sulfate - as MgSO4·7H2O - approximate bc. half of 38.2044
    'EX_nh4_e': 3.6748, # Ammonia - as NH4Cl - approximate (several sources and "side-effect"
    'EX_k_e': 20.82177, # Potassium - as KCl - approximate (several sources and "side-effect"
    'EX_na1_e': 0.0, # Sodium - as NaCl, Na2HPO4
    'EX_cl_e': 3.03888, # Chlorid - as NaCl, NH4Cl, CaCl2 - approximate price from NaCl
    'EX_o2_e': 0.0, # oxygen - no costs
}


In [10]:
file_name = "..\\Results\\2025-11-16_BayesOpt_iML1515_growth-cost_100it_round_1.json"
results_iML1515= JSON_deserialize_load_results((file_name), model_iML1515)

In [33]:
# get initial values
model_iML1515.medium = medium_iJO1366_reduced
model_iML1515.objective = results_iML1515["model objective"]
init_growth = model_iML1515.slim_optimize()
init_cost = calc_cost_tot(costs_iJO1366_reduced, medium_iJO1366_reduced).cpu().numpy().item()

### Figure 2AB
* growth_rate
* cost
* growth_cost
* is_pareto
* n_batch
* n_start
* n_candidates
* init_growth_rate
* init_cost

In [ ]:
dataname = "Figure2AB.csv"

"""Extract the relevant information and store as csv"""
# growth rate tensors
growth_np = results_iML1515["growth rate tensors"].cpu().numpy() # are positive
# cost tensors
cost_np = results_iML1515["cost tensors"].cpu().numpy() # are positive
# pareto optimal points
pareto_np = results_iML1515["is pareto"].cpu().numpy()

# Combine into a DataFrame
df = pd.DataFrame({
    "growth_rate": growth_np,
    "cost": cost_np,
    "growth_cost": growth_np/cost_np,
    "is_pareto": pareto_np,
    "n_batch": results_iML1515["n_iter"],
    "n_start" : results_iML1515["n_start"],
    "n_candidates" : results_iML1515["n_candidates"],
    "init_growth_rate": init_growth,
    "init_cost": init_cost,
})

# Save to CSV
df.to_csv(dataname, index = False)

### Figure 2C
* all medium components
* growth_cost
* n_batch
* n_start
* n_candidates

In [30]:
dataname = "Figure2C.csv"

"""Extract the relevant information and store as csv"""
# convert list of dictionaries to DataFrame
df = pd.DataFrame(results_iML1515["medium list"])

# Combine into a DataFrame
df["growth_cost"] = growth_np/cost_np
df["n_batch"] = results_iML1515["n_iter"]
df["n_start"] = results_iML1515["n_start"]
df["n_candidates"] = results_iML1515["n_candidates"]

# Save to CSV
df.to_csv(dataname, index = False)

# iJO1366 modified

In [31]:
model_iJO1366_antiEpEX_scFv = load_json_model("..\\iJO1366_producing_antiEpEX-scFv.json")

In [32]:
# M9 with essential trace metals and amino acids
medium_iJO1366_enriched = {
    'EX_pi_e': 34.90, # in M9
    'EX_mn2_e': 0.001, # - required?; drops at 0.0001
    'EX_fe2_e': 0.1, # - required?; drops at 0.01
    'EX_glc__D_e': 10.0, # in M9
    'EX_zn2_e': 0.001, # - required?; drops at 0.0001
    'EX_mg2_e': 1.0, # in M9 
    'EX_ca2_e': 0.05, # in M9
    'EX_ni2_e': 0.001, # - required?; drops at 0.0001
    'EX_cu2_e': 0.001, # - required?; drops at 0.0001
    'EX_cobalt2_e': 0.0001, # - required; drops at 0.00001 
    'EX_mobd_e': 0.0005, # - required?; drops at 0.0001 - higher than for iML1515
    'EX_so4_e': 1.0, # in M9
    'EX_nh4_e': 9.3475, # in M9 # lower than with 10
    'EX_k_e': 11.02, # in M9
    'EX_na1_e': 52.038, # in M9
    'EX_cl_e': 13.6755, # in M9
    'EX_o2_e': 20.0, # in M9 II - drops at 10
    'EX_arg__L_e': 4.75, # L-Arginine
    'EX_asn__L_e': 3.05, # L-Asparagine
    'EX_gln__L_e' : 4.95 # L-Glutamine
}
bounds_iJO1366_enriched = {
    'EX_pi_e': (0.0, 50),
    'EX_mn2_e': (0.001, 0.001), # fixed "trace" 
    'EX_fe2_e': (0.1, 0.1), # fixed "trace"
    'EX_glc__D_e': (1.0, 10),
    'EX_zn2_e': (0.001, 0.001), # fixed "trace"
    'EX_mg2_e': (0.0, 10),
    'EX_ca2_e': (0.0, 10),
    'EX_ni2_e': (0.001, 0.001), # fixed "trace"
    'EX_cu2_e': (0.001, 0.001), # fixed "trace"
    'EX_cobalt2_e': (0.0001, 0.0001), # fixed "trace"
    'EX_mobd_e': (0.0005, 0.0005), # fixed "trace"
    'EX_so4_e': (0.0, 10),
    'EX_nh4_e': (0.0, 10), # close to the default of 9.3475
    'EX_k_e': (0.0, 20),
    'EX_na1_e': (0.0, 100.0), # if fixed - can be set to 0
    'EX_cl_e': (0.0, 20),
    'EX_o2_e': (0, 20), # if fixed - can't be set to 10 or lower
    'EX_arg__L_e': (0.0, 10.0), # L-Arginine
    'EX_asn__L_e': (0.0, 10.0), # L-Asparagine
    'EX_gln__L_e' : (0.0, 10.0) # L-Glutamine
}
# costs are in £/mol
costs_iJO1366_enriched = {
    'EX_pi_e': 23.4234, # Phosphate - approximate (several sources)
    'EX_mn2_e': 0.0, #33.25, # Manganese - MnCl2·4H20
    'EX_fe2_e': 0.0, #37.5, # IronII - as iron sulfate FeSO4·7H2O
    'EX_glc__D_e': 7.7647236, # Glucose
    'EX_zn2_e': 0.0, #28.3, # Zinc - as Zn(CH3CHOOH)·H2O
    'EX_mg2_e': 19.1022, # Magnesium - as MgSO4·7H2O - approximate bc. half of 38.2044
    'EX_ca2_e': 18.08223, # Calcium - as CaCl2·2H2O
    'EX_ni2_e': 0.0, #53.24, # Nickel - as NiCl2·6H2O
    'EX_cu2_e': 0.0, #31.37, # Copper - CuCl2·2H2O
    'EX_cobalt2_e': 0.0, #114.39, # Cobalt - as CoCl2·6H2O
    'EX_mobd_e': 0.0, #184.12, # Molybdenum - molybdate NaMoO4·2H2O
    'EX_so4_e': 19.1022, # Sulfate - as MgSO4·7H2O - approximate bc. half of 38.2044
    'EX_nh4_e': 3.6748, # Ammonia - as NH4Cl - approximate (several sources and "side-effect"
    'EX_k_e': 20.82177, # Potassium - as KCl - approximate (several sources and "side-effect"
    'EX_na1_e': 0.0, # Sodium - as NaCl, Na2HPO4
    'EX_cl_e': 3.03888, # Chlorid - as NaCl, NH4Cl, CaCl2 - approximate price from NaCl
    'EX_o2_e': 0.0, # oxygen - no costs
    'EX_arg__L_e': 61.1442, # L-Arginine
    'EX_asn__L_e': 93.01248, # L-Asparagine
    'EX_gln__L_e' : 80.23086 # L-Glutamine
}


In [34]:
file_name = "..\\Results\\2025-11-16_BayesOpt_iJO1366_antiEpEX_scFv_growth-production-cost_100it_round_1.json"
results_iJO1366_gpc = JSON_deserialize_load_results((file_name), model_iJO1366_antiEpEX_scFv)

In [53]:
# get initial values
model_iJO1366_antiEpEX_scFv.medium = medium_iJO1366_enriched
model_iJO1366_antiEpEX_scFv.objective = results_iJO1366_gpc["model objective"]
biomass_rxn = model_iJO1366_antiEpEX_scFv.reactions.get_by_id("BIOMASS_Ec_iJO1366_core_53p95M")
# limit growth rate to 0.85
biomass_rxn.bounds = (0.0, 0.85)

solution = model_iJO1366_antiEpEX_scFv.optimize()
init_growth = solution.fluxes[results_iJO1366_gpc["biomass objective"]]
init_prod = solution.fluxes[results_iJO1366_gpc["production objective"]]
init_cost = calc_cost_tot(costs_iJO1366_enriched, medium_iJO1366_enriched).cpu().numpy().item()

print(init_growth, init_cost, init_prod, sep = "\n")

0.85
2210.8624803400003
0.021073837077251294


## Main Text Figures

### Figure 3B
* growth_rate
* cost
* production
* n_batch
* n_start
* n_candidates
* init_growth_rate
* init_cost
* init_production

In [49]:
dataname = "Figure3B.csv"

"""Extract the relevant information and store as csv"""
# growth rate tensors
growth_np = results_iJO1366_gpc["growth rate tensors"].cpu().numpy() # are positive
# cost tensors
cost_np = results_iJO1366_gpc["cost tensors"].cpu().numpy() # are positive
# pareto optimal points
production_np = results_iJO1366_gpc["production tensors"].cpu().numpy()

# Combine into a DataFrame
df = pd.DataFrame({
    "growth_rate": growth_np,
    "cost": cost_np,
    "production": production_np,
    "n_batch": results_iJO1366_gpc["n_iter"],
    "n_start" : results_iJO1366_gpc["n_start"],
    "n_candidates" : results_iJO1366_gpc["n_candidates"],
    "init_growth_rate": init_growth,
    "init_cost": init_cost,
    "init_production": init_prod
})

# Save to CSV
df.to_csv(dataname, index = False)

### Figure 3CD
* all medium components
* growth_rate
* cost
* growth_per_cost
* n_batch
* n_start
* n_candidates

In [45]:
dataname = "Figure3CD.csv"

"""Extract the relevant information and store as csv"""
# growth rate tensors
growth_np = results_iJO1366_gpc["growth rate tensors"].cpu().numpy() # are positive
# cost tensors
cost_np = results_iJO1366_gpc["cost tensors"].cpu().numpy() # are positive
# pareto optimal points
production_np = results_iJO1366_gpc["production tensors"].cpu().numpy()

# convert list of dictionaries to DataFrame
df = pd.DataFrame(results_iJO1366_gpc["medium list"])

# Combine into a DataFrame
df["growth rate"] = growth_np
df["cost"] = cost_np
df["production rate"] = production_np
df["n_batch"] = results_iJO1366_gpc["n_iter"]
df["n_start"] = results_iJO1366_gpc["n_start"]
df["is pareto"] = results_iJO1366_gpc["is pareto"]
df["n_batch"] = results_iJO1366_gpc["n_iter"]
df["n_start"] = results_iJO1366_gpc["n_start"]
df["n_candidates"] = results_iJO1366_gpc["n_candidates"]

# Save to CSV
df.to_csv(dataname, index = False)

## Supplementary

### Supplementary1 - Growth-Production

In [54]:
file_name = "..\\Results\\2025-11-16_BayesOpt_iJO1366_antiEpEX_scFv_growth-production_100it_round_1.json"
results_iJO1366_gp = JSON_deserialize_load_results((file_name), model_iJO1366_antiEpEX_scFv)

dataname = "SuppFigure1.csv"
"""Extract the relevant information and store as csv"""
# growth rate tensors
growth_np = results_iJO1366_gp["growth rate tensors"].cpu().numpy()
# production tensors
production_np = results_iJO1366_gp["production tensors"].cpu().numpy()
# pareto optimal points
pareto_np = results_iJO1366_gp["is pareto"].cpu().numpy()

# Combine into a DataFrame
n_rows = len(growth_np)
df = pd.DataFrame({
    "growth_rate": growth_np,
    "production" : production_np,
    "is_pareto": pareto_np.astype(bool),
    "n_batch" : results_iJO1366_gp["n_iter"],
    "n_start" : results_iJO1366_gp["n_start"],
    "n_candidates" : results_iJO1366_gp["n_candidates"],
    "init_growth_rate": init_growth,
    "init_production": init_prod
})

# Save to CSV
df.to_csv(dataname, index = False)

### Supplementary2 - Growth-Cost

In [57]:
file_name = "..\\Results\\2025-11-16_BayesOpt_iJO1366_antiEpEX_scFv_growth-cost_100it_round_1.json"
results_iJO1366_gc = JSON_deserialize_load_results((file_name), model_iJO1366_antiEpEX_scFv)

dataname = "SuppFigure2.csv"

"""Extract the relevant information and store as csv"""
# growth rate tensors
growth_np = results_iJO1366_gc["growth rate tensors"].cpu().numpy()
# cost tensors
cost_np = results_iJO1366_gc["cost tensors"].cpu().numpy()
# pareto optimal points
pareto_np = results_iJO1366_gc["is pareto"].cpu().numpy()


# Combine into a DataFrame
n_rows = len(growth_np)
df = pd.DataFrame({
    "growth_rate": growth_np,
    "cost": cost_np,
    "is_pareto": pareto_np.astype(bool),
    "n_batch" : results_iJO1366_gc["n_iter"],
    "n_start" : results_iJO1366_gc["n_start"],
    "n_candidates" : results_iJO1366_gc["n_candidates"],
    "init_growth_rate": init_growth,
    "init_cost": init_cost
})

# Save to CSV
df.to_csv(dataname, index = False)

### Supplementary3 - Production-Cost

In [58]:
file_name = "..\\Results\\2025-11-16_BayesOpt_iJO1366_antiEpEX_scFv_production-cost_100it_round_1.json"
results_iJO1366_pc = JSON_deserialize_load_results((file_name), model_iJO1366_antiEpEX_scFv)

dataname = "SuppFigure3.csv"

"""Extract the relevant information and store as csv"""
# cost tensors
cost_np = results_iJO1366_pc["cost tensors"].cpu().numpy()
# production tensors
production_np = results_iJO1366_pc["production tensors"].cpu().numpy()
# pareto optimal points
pareto_np = results_iJO1366_pc["is pareto"].cpu().numpy()


# Combine into a DataFrame
n_rows = len(cost_np)
df = pd.DataFrame({
    "cost": cost_np,
    "production" : production_np,
    "is_pareto": pareto_np.astype(bool),
    "n_batch" : results_iJO1366_pc["n_iter"],
    "n_start" : results_iJO1366_pc["n_start"],
    "n_candidates" : results_iJO1366_pc["n_candidates"],
    "init_cost": init_cost,
    "init_production" : init_prod
})

# Save to CSV
df.to_csv(dataname, index = False)

### Supplementary4 - Triple with fixed M9

In [50]:
file_name = "..\\Results\\2025-11-18_BayesOpt_iJO1366_antiEpEX_scFv_growth-production-cost_fixedM9_100it_round_1.json"
results_iJO1366_M9 = JSON_deserialize_load_results((file_name), model_iJO1366_antiEpEX_scFv)

dataname = "SuppFigure4.csv"

"""Extract the relevant information and store as csv"""
# growth rate tensors
growth_np = results_iJO1366_M9["growth rate tensors"].cpu().numpy() # are positive
# cost tensors
cost_np = results_iJO1366_M9["cost tensors"].cpu().numpy() # are positive
# pareto optimal points
production_np = results_iJO1366_M9["production tensors"].cpu().numpy()

# Combine into a DataFrame
df = pd.DataFrame({
    "growth_rate": growth_np,
    "cost": cost_np,
    "production": production_np,
    "n_batch": results_iJO1366_M9["n_iter"],
    "n_start" : results_iJO1366_M9["n_start"],
    "n_candidates" : results_iJO1366_M9["n_candidates"],
    "init_growth_rate": init_growth,
    "init_cost": init_cost,
    "init_production": init_prod
})

# Save to CSV
df.to_csv(dataname, index = False)